In [5]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install neattext
import neattext.functions as nfx
import matplotlib.pyplot as plt
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,Bidirectional,GlobalMaxPooling1D,Input,Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 112 kB 7.7 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=str(sent).lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
#         sent=nfx.remove_shortwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [7]:
import pandas as pd
reddit_train=pd.read_csv('/content/drive/My Drive/Reddit_data/Processed/Posts_train_clean.csv', lineterminator='\n')
reddit_train.text_cleaned[pd.isna(reddit_train['text_cleaned'])] = ''
cleaned_train_text_reddit,train_text_length_reddit=clean_text(pd.Series(reddit_train.title_cleaned+reddit_train.text_cleaned))



100%|██████████| 6366/6366 [00:00<00:00, 20686.79it/s]


In [8]:
reconstructed_model = keras.models.load_model("/content/drive/My Drive/suicide_detector_model.h5")

# loading
with open('/content/drive/My Drive/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [12]:
train_text_seq_reddit=tokenizer.texts_to_sequences(cleaned_train_text_reddit)
train_text_pad_reddit=pad_sequences(train_text_seq_reddit,maxlen=100)


In [13]:
predicted_train_reddit = reconstructed_model.predict(train_text_pad_reddit)

In [17]:
reddit_train.columns

Index(['Unnamed: 0', 'Author', 'Time', 'ID', 'Content', 'subreddit', 'Title',
       'text_cleaned', 'title_cleaned'],
      dtype='object')

In [18]:
dt = pd.concat([pd.Series(predicted_train_reddit.flatten()), reddit_train.Author, reddit_train.subreddit, reddit_train.Time], axis=1)
dt.columns = ["Score", "Author",  "subreddit", "Time"]

In [19]:
dt.to_csv("/content/drive/My Drive/Reddit_data/Processed/train_score.csv")